In [1]:
import sys
sys.path.append("/home/kelab/m6AAIpy2")

In [2]:
from keras.models import load_model
from pkg_resources import resource_filename
import numpy as np
import pandas as pd
from Bio.Seq import Seq
import keras.backend as kb

Using TensorFlow backend.


In [3]:
def one_hot_encode(seq):

    map = np.asarray([[0, 0, 0, 0],
                      [1, 0, 0, 0],
                      [0, 1, 0, 0],
                      [0, 0, 1, 0],
                      [0, 0, 0, 1]])

    seq = seq.upper().replace('A', '\x01').replace('C', '\x02')
    seq = seq.replace('G', '\x03').replace('T', '\x04').replace('N', '\x00')

    return map[np.fromstring(seq, np.int8) % 5]

In [4]:
def categorical_crossentropy_2d(y_true, y_pred):
    # Standard categorical cross entropy for sequence outputs

    return - kb.mean(y_true[:, :, 0]*kb.log(y_pred[:, :, 0]+1e-10)
                   + y_true[:, :, 1]*kb.log(y_pred[:, :, 1]+1e-10))

In [5]:
context = 10000

In [6]:
paths = ('/home/kelab/iM6A/mouseRAC10000_c{}.h5'.format(x) for x in range(1, 6))

In [7]:
models = [load_model(y, custom_objects={'categorical_crossentropy_2d': categorical_crossentropy_2d}) for y in paths]

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [8]:
models

### Read data

In [9]:
import numpy as np
import pandas as pd

In [10]:
Fasta = pd.read_csv("mm10_Fasta_Non_Donor_Acceptor_Replace.csv")

In [11]:
Fasta

,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds,...,LastIntronLength,SecondToLastLength,Pre,PreSequence,SecondToLastExonSequence,LastIntronSequence,LastExonSequence,Truncation,Seq,Replace
0,Pdcd11,chr19,+,47090765,47131143,47092798,47130810,36,"47090765,47092781,47093750,47095207,47096854,4...","47090867,47092900,47093882,47095375,47097016,4...",...,525,166,39182,AGAACGGAAGTAGCCCTGGTCTTACCTACGCTCGTGGACGAGTCGC...,ATGTGGATGTGATCGTCAAGTTTGCCCAGCTGGAGTTCCAGCTCGG...,GTGAGGGCGACAGCAGGGCTCATTCCTGTTGGAGTGTGGGCTGGGA...,GGACATCTTTGAGCGGGTCATTCACCTGAGCCTGGCGCCTAAGAGG...,GTGAGGGCGACAGCAGGGCTCATTCCTGTTGGAGTGTGGGCTGGGA...,ATGTGGATGTGATCGTCAAGTTTGCCCAGCTGGAGTTCCAGCTCGG...,CCACTGTCACTCACAGCACCAGCAGCCAGTCACAGATGGGGGCAAG...
1,Gna13,chr11,+,109362830,109401369,109362973,109396486,4,"109362830,109365535,109392345,109395913,","109363256,109365762,109392396,109401369,",...,3517,51,29515,AGTCCTCGGGTTCCCTCCCTCCCCGGGCGCGCTCGGGCCGCCGCTG...,GGTGAGTCTGTAAAGTATTTCTTGGATAACTTGGATAAACTTGGAG...,GTAAGTTTCAAAACAATAGTTCTGGTTTTGCCTTCAACAGTTGATA...,GATTACATTCCATCACAGCAAGATATCCTGCTTGCCAGAAGGCCCA...,GTAAGTTTCAAAACAATAGTTCTGGTTTTGCCTTCAACAGTTGATA...,GCAGAGTCTGTAAAGTATTTCTTGGATAACTTGGATAAACTTGGAG...,GCTCTTCTTGCCTAGCTTCAAGTCCCAGGCTGTGTATTTTTTTTCA...
2,Wscd2,chr5,+,113490446,113589725,113550934,113588145,9,"113490446,113550422,113558348,113560900,113570...","113490940,113551334,113558463,113561085,113570...",...,10329,201,86816,CACGCTGAACACCCATCAACAAAACTGTCCTTTGTTGGATTTCTTT...,GGTTCAAAGGTGAGCGGGACCACTGGCGAAGCGGGCGGACCATCTG...,GTGACAGGAGGGAGGGTCGTGGACAGTGTGAGGTGCGGGTAATCAG...,AGTGGCCCGAGTTCGTGAGGAACTATGCCCCGTGGTGGGCCACGCA...,GTGACAGGAGGGAGGGTCGTGGACAGTGTGAGGTGCGGGTAATCAG...,GGTTCAAAGCAGAGCGGGACCACTGGCGAAGCGGGCGGACCATCTG...,CCTTTGCAAAGGTCGGTTCTCTTCACTGAACAGCACTATAGCGAAC...
3,Trim80,chr11,+,115440544,115448270,115440553,115448220,8,"115440544,115440706,115441080,115445015,115445...","115440606,115440750,115441745,115445111,115445...",...,843,63,6213,AGAGAGACCATGAAACAGCGGCCACAGAGAAAGGAGAGAAGACGGC...,GAATCAGGATGAGCTCCTATGAGGTGCTGCCCGGAACTGTGGAGAG...,GTGAGTTCTCAAACCAGAGGCCAACCCTGGGGGAGGAGAGGAGGGA...,ATTACTGCAACCTGAACTTTGACCCCTGCACGGCCAGCGAAGAGCT...,GTGAGTTCTCAAACCAGAGGCCAACCCTGGGGGAGGAGAGGAGGGA...,GAATCAGGATGAGCTCCTATGAGGTGCTGCCCGGAACTGTGGAGAG...,AGCAGTTCCGCAATCCTTCAGGACCTGTCGTGGACTTACCCAGCAC...
4,Strbp,chr2,-,37569867,37647285,37584029,37647118,17,"37569867,37586509,37590656,37596703,37599120,3...","37584106,37586554,37590780,37596844,37599255,3...",...,2403,45,60731,GTGTATTCCAGAGAACAATTTTGAGATATCTTTGCTTATATTTGAC...,GTTACGGAACCCCATATGGTTACAGCACAGCTGCCCCTGCATATG,GTATGTACACTTCCTTACTGATTCCTCCTCTTAGCAAATTTGTCAC...,GTTTGCCCAAGAGAATGGTTCTGTTACCCGTTATGAAATTCCCAAC...,GTATGTACACTTCCTTACTGATTCCTCCTCTTAGCAAATTTGTCAC...,GTTACGGAACCCCATATGGTTACAGCACAGCTGCCCCTGCATATGC...,TCCCGTTTCCCCTCGGCGCTGACCTGGTGGCTGGTCCAGGCTGTGT...
5,Kl,chr5,+,150952606,150993817,150952716,150991854,5,"150952606,150980609,150982502,150988392,150991...","150953541,150981120,150982771,150989494,150993...",...,2022,1102,35786,GATAATCATTGCTCGTGGGGCGGCGGGAGCGGGGGTGGGCACCGCG...,GTGGACACTACTCTCTCTCAGTTTACTGACCCGAATGTCTATCTGT...,GTGAGACCCCGCCCCCTCAATCCCGGAAATACATCAGAGCGTGTAT...,CCTACGTGTTGGACGACATCAACCTTTGTGGCTACTTTGCGTATTC...,GTGAGACCCCGCCCCCTCAATCCCGGAAATACATCAGAGCGTGTAT...,GTGGACACTACTCTCTCTCAGTTTACTGACCCGAATGTCTATCTGT...,TCCAGCACCCACAATCTTCCTCGGATGAATCTTCTTCCATATAATT...
6,Tmx3,chr18,+,90510153,90543267,90510322,90540196,16,"90510153,90511278,90514204,90517393,90519104,9...","90510377,90511333,90514244,90517517,90519150,9...",...,1371,69,28345,CTTCGCTGGGCTACCGGGATCCTGAAGGAGGCCGGCCTAccccgcc...,GCCCAAGGAGGTGATAGCATTTTTCAGAGATTGAAAAGAATAGTAT...,GTAAGTTCTGACAGTATCTTCAATAGCTGAAAAATTTGGGGTTTTC...,TCTATATTCAAGAGCTCTCCACTTATGGGCTGCTTTCTTTTCGGGC...,GTAAGTTCTGACAGTATCTTCAATAGCTGAAAAATTTGGGGTTTTC...,GCCCAAGGAGCAGATAGCATTTTTCAGAGATTGAAAAGAATAGTAT...,GAGAACCCAGGCTCCGGGTCTCTCTGGGCAGTAGGGGGCGGCGCAG...
7,Arhgap21,chr2,-,20847918,20967721,20848711,20967410,25,"20847918,20853037,20854934,20855353,20855580,2.

### Select positive strand

In [12]:
Fasta_Pos = Fasta[Fasta["strand"]=="+"]

In [13]:
Fasta_Pos = Fasta_Pos.reset_index(drop = True)

In [14]:
Fasta_Pos

,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds,...,LastIntronLength,SecondToLastLength,Pre,PreSequence,SecondToLastExonSequence,LastIntronSequence,LastExonSequence,Truncation,Seq,Replace
0,Pdcd11,chr19,+,47090765,47131143,47092798,47130810,36,"47090765,47092781,47093750,47095207,47096854,4...","47090867,47092900,47093882,47095375,47097016,4...",...,525,166,39182,AGAACGGAAGTAGCCCTGGTCTTACCTACGCTCGTGGACGAGTCGC...,ATGTGGATGTGATCGTCAAGTTTGCCCAGCTGGAGTTCCAGCTCGG...,GTGAGGGCGACAGCAGGGCTCATTCCTGTTGGAGTGTGGGCTGGGA...,GGACATCTTTGAGCGGGTCATTCACCTGAGCCTGGCGCCTAAGAGG...,GTGAGGGCGACAGCAGGGCTCATTCCTGTTGGAGTGTGGGCTGGGA...,ATGTGGATGTGATCGTCAAGTTTGCCCAGCTGGAGTTCCAGCTCGG...,CCACTGTCACTCACAGCACCAGCAGCCAGTCACAGATGGGGGCAAG...
1,Gna13,chr11,+,109362830,109401369,109362973,109396486,4,"109362830,109365535,109392345,109395913,","109363256,109365762,109392396,109401369,",...,3517,51,29515,AGTCCTCGGGTTCCCTCCCTCCCCGGGCGCGCTCGGGCCGCCGCTG...,GGTGAGTCTGTAAAGTATTTCTTGGATAACTTGGATAAACTTGGAG...,GTAAGTTTCAAAACAATAGTTCTGGTTTTGCCTTCAACAGTTGATA...,GATTACATTCCATCACAGCAAGATATCCTGCTTGCCAGAAGGCCCA...,GTAAGTTTCAAAACAATAGTTCTGGTTTTGCCTTCAACAGTTGATA...,GCAGAGTCTGTAAAGTATTTCTTGGATAACTTGGATAAACTTGGAG...,GCTCTTCTTGCCTAGCTTCAAGTCCCAGGCTGTGTATTTTTTTTCA...
2,Wscd2,chr5,+,113490446,113589725,113550934,113588145,9,"113490446,113550422,113558348,113560900,113570...","113490940,113551334,113558463,113561085,113570...",...,10329,201,86816,CACGCTGAACACCCATCAACAAAACTGTCCTTTGTTGGATTTCTTT...,GGTTCAAAGGTGAGCGGGACCACTGGCGAAGCGGGCGGACCATCTG...,GTGACAGGAGGGAGGGTCGTGGACAGTGTGAGGTGCGGGTAATCAG...,AGTGGCCCGAGTTCGTGAGGAACTATGCCCCGTGGTGGGCCACGCA...,GTGACAGGAGGGAGGGTCGTGGACAGTGTGAGGTGCGGGTAATCAG...,GGTTCAAAGCAGAGCGGGACCACTGGCGAAGCGGGCGGACCATCTG...,CCTTTGCAAAGGTCGGTTCTCTTCACTGAACAGCACTATAGCGAAC...
3,Trim80,chr11,+,115440544,115448270,115440553,115448220,8,"115440544,115440706,115441080,115445015,115445...","115440606,115440750,115441745,115445111,115445...",...,843,63,6213,AGAGAGACCATGAAACAGCGGCCACAGAGAAAGGAGAGAAGACGGC...,GAATCAGGATGAGCTCCTATGAGGTGCTGCCCGGAACTGTGGAGAG...,GTGAGTTCTCAAACCAGAGGCCAACCCTGGGGGAGGAGAGGAGGGA...,ATTACTGCAACCTGAACTTTGACCCCTGCACGGCCAGCGAAGAGCT...,GTGAGTTCTCAAACCAGAGGCCAACCCTGGGGGAGGAGAGGAGGGA...,GAATCAGGATGAGCTCCTATGAGGTGCTGCCCGGAACTGTGGAGAG...,AGCAGTTCCGCAATCCTTCAGGACCTGTCGTGGACTTACCCAGCAC...
4,Kl,chr5,+,150952606,150993817,150952716,150991854,5,"150952606,150980609,150982502,150988392,150991...","150953541,150981120,150982771,150989494,150993...",...,2022,1102,35786,GATAATCATTGCTCGTGGGGCGGCGGGAGCGGGGGTGGGCACCGCG...,GTGGACACTACTCTCTCTCAGTTTACTGACCCGAATGTCTATCTGT...,GTGAGACCCCGCCCCCTCAATCCCGGAAATACATCAGAGCGTGTAT...,CCTACGTGTTGGACGACATCAACCTTTGTGGCTACTTTGCGTATTC...,GTGAGACCCCGCCCCCTCAATCCCGGAAATACATCAGAGCGTGTAT...,GTGGACACTACTCTCTCTCAGTTTACTGACCCGAATGTCTATCTGT...,TCCAGCACCCACAATCTTCCTCGGATGAATCTTCTTCCATATAATT...
5,Tmx3,chr18,+,90510153,90543267,90510322,90540196,16,"90510153,90511278,90514204,90517393,90519104,9...","90510377,90511333,90514244,90517517,90519150,9...",...,1371,69,28345,CTTCGCTGGGCTACCGGGATCCTGAAGGAGGCCGGCCTAccccgcc...,GCCCAAGGAGGTGATAGCATTTTTCAGAGATTGAAAAGAATAGTAT...,GTAAGTTCTGACAGTATCTTCAATAGCTGAAAAATTTGGGGTTTTC...,TCTATATTCAAGAGCTCTCCACTTATGGGCTGCTTTCTTTTCGGGC...,GTAAGTTCTGACAGTATCTTCAATAGCTGAAAAATTTGGGGTTTTC...,GCCCAAGGAGCAGATAGCATTTTTCAGAGATTGAAAAGAATAGTAT...,GAGAACCCAGGCTCCGGGTCTCTCTGGGCAGTAGGGGGCGGCGCAG...
6,Gem,chr4,+,11704456,11714752,11705913,11713753,5,"11704456,11705904,11711138,11711969,11713475,","11704566,11706241,11711215,11712174,11714752,",...,1301,205,7513,CTGCCAGCGGACATCACACAGCCTCGGACTGCGGAGCCCGCATCGC...,GGGGAGAACGAATGGCTCCATGACCACTGCATGCAGGTCGGGGATG...,GTGAGAGGGCGTGTGACATGGGCTCTACCCCTGCATGGAACACATT...,AAGGGAGAGCTTGTGCTGTGGTGTTCGACTGCAAATTCATCGAGAC...,GTGAGAGGGCGTGTGACATGGGCTCTACCCCTGCATGGAACACATT...,GGGGAGAACGAATGGCTCCATGACCACTGCATGCAGCACGGGGATG...,TGAGTTTTGGGCCTAGGTATGGGTTTGTGTATGAGCTGGAAACAGG...
7,Ky,chr9,+,102505749,102546239,102506142,102542781,11,"102505749,102509110,102512972,102521288,102523...","1025

In [15]:
PreSequence = Fasta_Pos["PreSequence"].tolist()
Sequence = Fasta_Pos["Seq"].tolist()
ReplaceSequence = Fasta_Pos["Replace"].tolist()

In [16]:
df = pd.DataFrame(range(-1000,1001),columns=["index"])

In [17]:
for i in range(len(Fasta_Pos)):
    
    # Define exon location   
    Exon = []
    a = Fasta_Pos.loc[i, "exonStarts"].split(",")[0:-1]
    b = Fasta_Pos.loc[i, "exonEnds"].split(",")[0:-1]
    A = [int(u) for u in a]
    B = [int(v) for v in b]
    Exon = A + B
    Exon = sorted(Exon)
    
    # Define length of exon and intron
    Length = []
    for j in range(1,len(Exon),1):
        length = Exon[j] - Exon[-1 + j]
        Length.append(length)
    
    Length = Length[:-3]
    Length.append(Fasta_Pos.loc[i,"SecondToLastLength"] + 100 + Fasta_Pos.loc[i,"LastExonLength"])
    
    # transcript
    Pre = PreSequence[i]
    Seq = Sequence[i]
    Replace = ReplaceSequence[i]
    
    SecondToLastExon = Seq[0:(Fasta_Pos.loc[i,"SecondToLastLength"])]
    Intron = Seq[(Fasta_Pos.loc[i,"SecondToLastLength"]):(Fasta_Pos.loc[i,"SecondToLastLength"]+100)]
    LastExon = Seq[-(Fasta_Pos.loc[i,"LastExonLength"]):]   
    
    input_sequence = Pre + SecondToLastExon + Replace + LastExon    
    
    ## Predict transcript
    x = one_hot_encode('N'*(context//2) + input_sequence + 'N'*(context//2))[None, :]
    y = np.mean([models[m].predict(x) for m in range(5)], axis=0)
    m6AAI_prob = y[0, :, 1]
    m6AAI_prob = m6AAI_prob.tolist()    
    
    # Define prob in exons
    CumSum = []
    Sum = 0
    for k in Length:
        Sum = Sum + k
        CumSum.append(Sum)    
    
    First_prob = [m6AAI_prob[0:CumSum[0]]]
    for m in range(1,len(CumSum),1):
        prob = m6AAI_prob[(CumSum[m-1]):(CumSum[m])]
        First_prob.append(prob)    
 
    Probability = []
    for n in range(0,len(First_prob),2):
        List = First_prob[n]
        Probability.append(List)

    iM6A_prob = []
    for t in Probability:
        iM6A_prob = iM6A_prob + t
        
    Start = 0
    for t in range(len(Probability)-1):
        Start = Start + len(Probability[t])        
    
    Pre = iM6A_prob[0:Start]
    Last = iM6A_prob[Start:]
    Last = Last[0:(Fasta_Pos.loc[i,"SecondToLastLength"])] + Last[-(Fasta_Pos.loc[i,"LastExonLength"]):]
    
    if len(Pre) < 1000:
        Pre = [0]*(1000-Start) + Pre
    if len(Pre) >= 1000:
        Pre = Pre[-1000:]
    
    if len(Last) < 1001:
        Last = Last + [0]*(1001-len(Last))
    if len(Last) >= 1001:
        Last = Last[0:1001]
    
    New = np.array(Pre + Last)
    New = pd.DataFrame({Fasta_Pos.loc[i,"name"]:New})
    
    df = pd.merge(df, New, left_index=True, right_index=True)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  if sys.path[0] == '':


### Select negative strand

In [18]:
Fasta_Neg = Fasta[Fasta["strand"]=="-"]

In [19]:
Fasta_Neg = Fasta_Neg.reset_index(drop = True)

In [20]:
PreSequence = Fasta_Neg["PreSequence"].tolist()
Sequence = Fasta_Neg["Seq"].tolist()
ReplaceSequence = Fasta_Neg["Replace"].tolist()

In [21]:
for i in range(len(Fasta_Neg)):
    
    # Define exon location
    Exon = []
    a = Fasta_Neg.loc[i, "exonStarts"].split(",")[0:-1]
    b = Fasta_Neg.loc[i, "exonEnds"].split(",")[0:-1]
    A = [int(u) for u in a]
    B = [int(v) for v in b]
    Exon = A + B
    Exon = sorted(Exon)
    
    # Define length of exon and intron
    Length = []
    for j in range(1,len(Exon),1):
        length = Exon[j] - Exon[-1 + j]
        Length.append(length)
    Length = Length[::-1]
    
    Length = Length[:-3]
    Length.append(Fasta_Neg.loc[i,"SecondToLastLength"] + 100 + Fasta_Neg.loc[i,"LastExonLength"])
    
    # transcript
    Pre = PreSequence[i]
    Seq = Sequence[i]
    Replace = ReplaceSequence[i]
    
    SecondToLastExon = Seq[0:(Fasta_Neg.loc[i,"SecondToLastLength"])]
    Intron = Seq[(Fasta_Neg.loc[i,"SecondToLastLength"]):(Fasta_Neg.loc[i,"SecondToLastLength"]+100)]
    LastExon = Seq[-(Fasta_Neg.loc[i,"LastExonLength"]):]   
    
    input_sequence = Pre + SecondToLastExon + Replace + LastExon
    
    ## Predict transcript
    x = one_hot_encode('N'*(context//2) + input_sequence + 'N'*(context//2))[None, :]
    y = np.mean([models[m].predict(x) for m in range(5)], axis=0)
    m6AAI_prob = y[0, :, 1]
    m6AAI_prob = m6AAI_prob.tolist()    
    
    CumSum = []
    Sum = 0
    for k in Length:
        Sum = Sum + k
        CumSum.append(Sum)

    # Define prob in exons
    First_prob = [m6AAI_prob[0:CumSum[0]]]
    for m in range(1,len(CumSum),1):
        prob = m6AAI_prob[(CumSum[m-1]):(CumSum[m])]
        First_prob.append(prob)
    
    Probability = []
    for n in range(0,len(First_prob),2):
        List = First_prob[n]
        Probability.append(List)

    iM6A_prob = []
    for t in Probability:
        iM6A_prob = iM6A_prob + t
        
    Start = 0
    for t in range(len(Probability)-1):
        Start = Start + len(Probability[t])        
    
    Pre = iM6A_prob[0:Start]
    Last = iM6A_prob[Start:]
    Last = Last[0:(Fasta_Neg.loc[i,"SecondToLastLength"])] + Last[-(Fasta_Neg.loc[i,"LastExonLength"]):]
    
    if len(Pre) < 1000:
        Pre = [0]*(1000-Start) + Pre
    if len(Pre) >= 1000:
        Pre = Pre[-1000:]
    
    if len(Last) < 1001:
        Last = Last + [0]*(1001-len(Last))
    if len(Last) >= 1001:
        Last = Last[0:1001]
    
    New = np.array(Pre + Last)
    New = pd.DataFrame({Fasta_Neg.loc[i,"name"]:New})
    
    df = pd.merge(df, New, left_index=True, right_index=True)            

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  if sys.path[0] == '':


KeyError: u'the label [1157] is not in the [index]'

In [ ]:
df

In [ ]:
df.drop(["index"], axis=1, inplace=True)
df = df.T

In [ ]:
df.loc["Sum"] = df.sum()
df = df.T

In [ ]:
Value = df[["Sum"]]

In [ ]:
Value.head(5)

In [ ]:
df.drop(["Sum"], axis=1, inplace=True)
df = df.T

In [ ]:
df.loc["Number"] = (df > 0).sum()
df = df.T

In [ ]:
Number = df[["Number"]]

In [ ]:
Number.head(5)

In [ ]:
Data = pd.concat([Value, Number], axis=1)

In [ ]:
Data["Mean"] = Data["Sum"]/Data["Number"]

In [ ]:
Data = Data.head(2000)
Sum = []
Number = []

for j in range(0,len(Data),10):

    a = Data.loc[j:(j+9),"Sum"]
    b = np.mean(a, axis=0)
    Sum.append(b)
    
    a = Data.loc[j:(j+9),"Number"]
    b = np.mean(a, axis=0)
    Number.append(b)

Result = pd.DataFrame({"Index":range(-1000,1000,10), "Sum":Sum, "Number":Number})
Result["Mean"] = Result["Sum"]/Result["Number"] 

In [ ]:
Result.to_csv("iM6A_Mouse_NewLastExonStart_TruncationWithSpliceMutationReplace_WithoutIntron.csv", index=0)